Simple Vanilla Neural Net 

In [ ]:
#Image classifer with logistic regression
#images shape(width,height,3(rgb))

import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
import math
from scipy import ndimage

#from basicFunctions import crossEntropyLoss 
 

In [ ]:
def crossEntropyLoss(a,Y):
    num_of_examples = Y.shape[1] #len(Y) doesn't work, need 2nd dimesnion
    a = a.T
    loss = - (1 / num_of_examples) * np.sum(np.dot((1 - Y), np.log(1 - a)) + np.dot(Y, np.log(a)))
    return loss

In [ ]:
# Testing cross entropy function
test_y = np.asarray([[1,1,1]])
test_x = np.asarray([[.8,.9,.4]])
print(crossEntropyLoss(test_x,test_y))


Expected output = [1231. 12312. 1231. ]

In [169]:
def sigmoid(x):
    output = 1.0/(1.0+np.exp(-x))
    cached_value = x
    cached_value = {"out_a":x}
    return output, cached_value

In [168]:
def relu(x,leaky=0):
    output = np.maximum(leaky,x)
#     cached_value = x
    cached_value = {"out_a":x}
    return output, cached_value
    

In [ ]:
# To generate numbers randomly with numpy, we will use np.random.randn(dimension_x, dimension_y)

In [ ]:
def initialize_params_shallow(input_layer, hidden_layer, output_layer):
    w1 = np.random.randn(hidden_layer,input_layer)
    w2 = np.random.randn(output_layer,hidden_layer)
    #np.random.randn gives numbers larger than 1 so: (vanishing/exploading gradients)
    w1,w2 = w1 * .01, w2 * .01
    
    #is it by one because of broadcasting 
    b1 = np.zeros((hidden_layer,1))
    b2 = np.zeros((output_layer,1))
    
    assert(w1.shape == (hidden_layer, input_layer))
    assert(w2.shape == (output_layer, hidden_layer))
    assert(b1.shape == (hidden_layer,1))
    assert(b2.shape == (output_layer,1))
    
    return {"w1":w1,"w2":w2,"b1":b1,"b2":b2}

In [ ]:
params_shallow = initialize_params_shallow(3, 4, 2)
print("w1:", params_shallow["w1"])
print("w2:", params_shallow["w2"])
print("b1:", params_shallow["b1"])
print("b2:", params_shallow["b2"])

In [112]:
def initialize_params_deep(input_layer, hidden_layers, output_layer):
#     np.random.seed(3) #non random random numbers (will get the same random numbers each time) used for testing
    layers = {} #dictionary of weights and biasis
    previous_layer = input_layer #sets the begining shape as shape of input layer
    for i in range(len(hidden_layers)):
        cur_layer = hidden_layers[i]
        weights = np.random.randn(cur_layer,previous_layer)
        weights *= .01 #because np.random.rand returns numbers too large, more efficeint durring gradient descensed 
        bias = np.zeros((cur_layer,1))
        cur_weights_key = "w" + str(i + 1) # i + 1 because humans like things indexed at 0
        cur_bias_key = "b" + str(i + 1)
        layers[cur_weights_key] = weights
        layers[cur_bias_key] = bias 
        
        assert(layers["w" + str(i + 1)].shape == (cur_layer, previous_layer))
        assert(layers["b" + str(i + 1)].shape == (cur_layer,1))
        
        previous_layer = cur_layer

    layers["w" + str(len(hidden_layers) + 1)] = np.random.randn(output_layer,previous_layer) * .01
    layers["b" + str(len(hidden_layers) + 1)] = np.zeros((output_layer,1))

    return layers

In [113]:
params_deep1 = initialize_params_deep(5,[4],3)
for param in params_deep2:
    print(param, params_deep2[param].shape)


w1 (6, 5)
b1 (6, 1)
w2 (7, 6)
b2 (7, 1)
w3 (10, 7)
b3 (10, 1)
w4 (3, 10)
b4 (3, 1)


In [ ]:
params_deep2 = initialize_params_deep(5,[6,7,10],3)
for param in params_deep2:
    print(param, params_deep2[param].shape)


In [170]:
def linear_forward_prop(a,w,b): #a because a is from the past layer
    w = w.T
    a = a.T
    z = np.dot(a,w) + b
#     print("z",z.shape,"a",a.shape[0],"w",w.shape[1],"b",b.shape)
    assert(z.shape == (a.shape[0], w.shape[1] ))
    #     assert(z.shape == np.dot(a,w).shape)
    cached_inputs = {"in_a":a,"w":w,"b":b} #(a,w,b)
    return z,cached_inputs
    

In [141]:
np.random.seed(1)
X = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1) #(1,1) broadcasting

X = np.array(X)
w = np.array(w)
b = np.array(b)
z,_ = linear_forward_prop(X,w,b)
print("X",X.shape,"w",w.shape,"b",b.shape,"z",z.shape)
print(_)

X (3, 2) w (1, 3) b (1,) z (2, 1)
{'a': array([[ 1.62434536, -0.52817175,  0.86540763],
       [-0.61175641, -1.07296862, -2.3015387 ]]), 'w': array([[ 1.74481176],
       [-0.7612069 ],
       [ 0.3190391 ]]), 'b': array([-0.24937038])}


In [164]:
def singleFowardPropagation(X,w,b,activationType,leaky=0): #TODO change x to a
#     cached_inputs = {"w":None,"x":None,"b":None,"z":None}
    z,cached_values = linear_forward_prop(X,w,b)
    if activationType == "sigmoid":
        a, activation_cache = sigmoid(z)
    if activationType == "relu":
        a, activation_cache = relu(z,leaky=leaky)
#     final_cache = c
    cached_values.update(activation_cache)
    return a, cached_values  #add asserts 

In [173]:
np.random.seed(2)
X = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1) #(1,1) broadcasting
a,cached_values = singleFowardPropagation(X,w,b,"relu")
print("A",a)
print("cached_values",cached_values)
# print(X,w,b)

A [[ 3.43896131]
 [ 0.        ]]
cached_values {'in_a': array([[-0.41675785, -2.1361961 , -1.79343559],
       [-0.05626683,  1.64027081, -0.84174737]]), 'w': array([[ 0.50288142],
       [-1.24528809],
       [-1.05795222]]), 'b': array([[-0.90900761]]), 'out_a': array([[ 3.43896131],
       [-2.08938436]])}


In [144]:
def fowardPropagate(X,params):
    num_layers = len(params)/2
    a = X
    cached_values = []
    for i in range(1,num_layers):
        w = params["w" + str(i)]
        b = params["b" + str(i)]
        past_a = a
        a, cur_cached = singleFowardProgagation(past_a,w,b,"relu")
        cached_values += [cur_cached] #TODO: get foramt of cached values 
    w = params["w" + len(num_layers)]
    b = params["b" + len(num_layers)]
    a, cur_cached = singleFowardProgagation(a,w,b,"sigmoid")
    cached_values += [cur_cached] 
    #write assert statemt here
    #get shape of a at the end 
    
    return a, cached_values